In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
train = pd.read_csv('train_NIR5Yl1.csv')
train.head()

,ID,Tag,Reputation,Answers,Username,Views,Upvotes
0,52664,a,3942.0,2.0,155623,7855.0,42.0
1,327662,a,26046.0,12.0,21781,55801.0,1175.0
2,468453,c,1358.0,4.0,56177,8067.0,60.0
3,96996,a,264.0,3.0,168793,27064.0,9.0
4,131465,c,4271.0,4.0,112223,13986.0,83.0


In [4]:
test = pd.read_csv('test_8i3B3FC.csv')
test.head()

,ID,Tag,Reputation,Answers,Username,Views
0,366953,a,5645.0,3.0,50652,33200.0
1,71864,c,24511.0,6.0,37685,2730.0
2,141692,i,927.0,1.0,135293,21167.0
3,316833,i,21.0,6.0,166998,18528.0
4,440445,i,4475.0,10.0,53504,57240.0


In [5]:
train = train.drop(train[train.Views > 3000000].index)

In [6]:
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
le = LabelEncoder()

In [7]:
train['Tag'] = le.fit_transform(train['Tag'])

In [8]:
train.drop(['ID','Username'], axis=1,inplace =True)
target = train['Upvotes']

In [9]:
from sklearn.preprocessing import Binarizer
bn = Binarizer(threshold=7)

In [10]:
pd_watched = bn.transform([train['Answers']])[0]
train['pd_watched'] = pd_watched

In [11]:
from sklearn.preprocessing import StandardScaler
feature_names = [x for x in train.columns if x not in ['Upvotes']]

In [12]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(train[feature_names], target,test_size = 0.22,random_state =205)
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_val = sc.transform(x_val)

In [13]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model
poly = PolynomialFeatures(degree = 4,interaction_only=False, include_bias=True)
x_poly = poly.fit_transform(x_train)
poly.fit(x_train, y_train)

PolynomialFeatures(degree=4, include_bias=True, interaction_only=False,
                   order='C')

In [14]:
lin_reg = linear_model.LassoLars(alpha=0.021,max_iter=150)
lin_reg.fit(x_poly, y_train)

LassoLars(alpha=0.021, copy_X=True, eps=2.220446049250313e-16,
          fit_intercept=True, fit_path=True, max_iter=150, normalize=True,
          positive=False, precompute='auto', verbose=False)

In [16]:
pred_val = lin_reg.predict(poly.fit_transform(x_val))

In [17]:
from sklearn.metrics import r2_score
print(r2_score(y_val, pred_val))

0.9034378793327961


In [18]:
test = pd.read_csv('test_8i3B3FC.csv')
ids = test['ID']
test.drop(['ID','Username'], axis=1,inplace =True)

In [19]:
le = LabelEncoder()
test['Tag'] = le.fit_transform(test['Tag'])

In [20]:
bn = Binarizer(threshold=7)
pd_watched = bn.transform([test['Answers']])[0]
test['pd_watched'] = pd_watched

In [21]:
test = sc.fit_transform(test)

In [23]:
pred_test = lin_reg.predict(poly.fit_transform(test))
pred_test=abs(pred_test)

In [14]:
submission = pd.DataFrame({'ID': ids,
                           'Upvotes':pred_test
                           })

submission.to_csv("Solution.csv",index=False)